<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


## Threads, Processes, and the Global Interpreter Lock

There are two main ways to achieve single-machine parallelism in python: multiple threads or multiple processes.
The best choice depends on the type of work you're doing and Python's GIL (global interpreter lock).

The GIL is an implementation detail of CPython. It's a lock deep inside your python process that limits things so *only one thread in your process can be running python code at once*. Let's see an example, using `concurrent.futures` to compute the Fibonnaci numbers in parallel.

In [ ]:
import concurrent.futures

def fib(n):
    """Compute the `n`th fibonnaci number.
    
    This is a deliberatly slow, CPU intensive implemenation.
    """
    if n < 2:
        return n
    return fib(n - 2) + fib(n - 1)

In [ ]:
%time fib(34)

One `fib(34)` takes my machine about 2 seconds. Computing it 4 time should then take about 8 seconds.

In [ ]:
results = map(fib, [34, 34, 34, 34])
%time _ = list(results)

Let's compute it in parallel. This is embarrassingly parallel, so with 4 threads we *should* be back down to 2 seconds again.

In [ ]:
thread_pool = concurrent.futures.ThreadPoolExecutor(max_workers=4)

with concurrent.futures.ProcessPoolExecutor(max_workers=4) as pool:
    results = thread_pool.map(fib, [34, 34, 34, 34])
    %time _ = list(results)

It's actually *slower*! `concurrent.futures` (and Dask) make it easy to swap the compute backend between threads and processes.

## Exercise: Parallelize `fib` with Processes

Use a `concurrent.futures.ProcessPoolExecutor` to achieve the same task. Time how long it take.
See https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ProcessPoolExecutor, which has the same API as our `ThreadPoolExecutor`.

In [ ]:
# Your solution


In [ ]:
%load solutions/04-schedulers-fib-process.py

So, why use threads at all, if they can't actually run Python code in parallel? Because much of the code you're running isn't Python code. The performance-sensitive parts of libraries like NumPy and Pandas are written in C or Cython. Wherever possible, these libraries release the GIL. The standard library does this in places too, like when you make an HTTP request.

When the GIL isn't a concern (NumPy and pandas), we tend to prefer threads to avoid data serialization.
To do an operation on a pandas DataFrame in multiple processes, the data has to be serialized (using e.g. pickle) in  the first process and deserialized in the second process. This takes time and memory. Threads don't have serialization overhead.

David Beazley has some nice materials on the GIL: http://www.dabeaz.com/GIL/

# Schedulers

In the previous notebooks, we used `dask.delayed` and `dask.dataframe` to parallelize computations.
These work by building a *task graph* instead of executing immediately.
Each *task* represents some function to call on some data, and the full *graph* is the relationship between all the tasks.

When we wanted the actual result, we called `compute`, which handed the task graph off to a *scheduler*.

**Schedulers are responsible for running a task graph and producing a result**.

![](https://raw.githubusercontent.com/dask/dask-org/master/images/grid_search_schedule.gif)

Dask includes two types of schedulers.

First, there are the single machine schedulers that execute things in parallel using threads or processes (or synchronously for debugging). These are what we've used up until now.

Second, there's the Distributed scheduler, which is newer and has more features than the single machine scheduler. We'll discuss the advanced features [in part 6](06-distributed-advanced.ipynb). For now we'll introduce the distributed scheduler. Despite the name, the distributed scheduler works just fine on a single machine. We'll often recommend using it instead of the single-machine scheduler for the richer diagnostics.

## Distributed Scheduler

The `dask.distributed` system is composed of a single centralized scheduler and many worker processes. [Deploying](http://dask.pydata.org/en/latest/setup.html) a remote Dask cluster involves some additional effort. But doing things locally is just involves creating a `Client` object, which lets you interact with the "cluster" (local threads or processes on your machine).

In [ ]:
from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
client = Client()
client

Be sure to click the `Dashboard` link to open up the diagnostics dashboard.

In [ ]:
import os

import dask
import dask.dataframe as dd

df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})
largest_delay = df.DepDelay.max()

By default, creating a `Client` makes it the default scheduler. Any calls to `.compute` will use the cluster your `client` is attached to (See http://dask.pydata.org/en/latest/scheduling.html for how to specify which scheduler to use).

In [ ]:
%time largest_delay.compute().max()

---

### Exercise

Run the following computations while looking at the diagnostics page. In each case what is taking the most time?

In [ ]:
# Number of flights
_ = len(df)

In [ ]:
# Number of non-cancelled flights
_ = len(df[~df.Cancelled])

In [ ]:
# Number of non-cancelled flights per-airport
_ = df[~df.Cancelled].groupby('Origin').Origin.count().compute()

In [ ]:
# Average departure delay from each airport?
_ = df[~df.Cancelled].groupby('Origin').DepDelay.mean().compute()

In [ ]:
# Average departure delay per day-of-week
_ = df.groupby(df.Date.dt.dayofweek).DepDelay.mean().compute()

---

As a note, the `dask.distributed` scheduler supports and expands upon the `concurrent.futures` API.
This means you can take your `concurrent.futures`-compatible code and scale it to a cluster by using `dask.distributed` instead of `concurrent.futures`.

In [ ]:
results = client.map(fib, [34, 34, 34, 34])
%time _ = list(results)

The [Advanced Distributed](06-distributed-advanced.ipynb) notebook will discuss this (and how `dask.distributed` enhances the `concurrent.futures` API) in more detail.